In [ ]:
# Initialization
####################################
!pip install transformers

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found GPU at: /device:GPU:0
Sat Jan 28 09:27:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    51W / 400W |    630MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+--------

In [ ]:
# Configs
####################################

### Dataset config
classes5=['CWE-89', 'CWE-79', 'CWE-20', 'CWE-264', 'CWE-200']
classes10=['CWE-787', 'CWE-79', 'CWE-125', 'CWE-20', 'CWE-78', 'CWE-89', 'CWE-416', 'CWE-22', 'CWE-352', 'CWE-190']
classes15=['CWE-79', 'CWE-119', 'CWE-20', 'CWE-200', 'CWE-89', 'CWE-264', 'CWE-787', 'CWE-22', 'CWE-125', 'CWE-352', 'CWE-94', 'CWE-399', 'CWE-310', 'CWE-416', 'CWE-189']
classes20=['CWE-79', 'CWE-119', 'CWE-20', 'CWE-200', 'CWE-89', 'CWE-264', 'CWE-787', 'CWE-22', 'CWE-125', 'CWE-352', 'CWE-94', 'CWE-399', 'CWE-310', 'CWE-287', 'CWE-416', 'CWE-269', 'CWE-78', 'CWE-190', 'CWE-476', 'CWE-189']

classes = classes20;
datasetUrl = 'https://raw.githubusercontent.com/SamanShafigh/vulnerability_report_classification/main/nvd_dataset/top20_vul_dataset_nvd_only.csv';

### Training config
epochs = 20

### **Data Preparation**

In [ ]:
# Load dataset
####################################

df = pd.read_csv(datasetUrl, sep='\t')
numberOfClasses = len(classes)
print(numberOfClasses)

df.head()
df.info()
df.drop(len(df)-1, inplace=True)
df['type'] = df['type'].astype(int)
df['type'].value_counts()

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


# Define train & val dataset
####################################
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['report'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

print("generate training data")
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer) 
labels = np.zeros((len(df), numberOfClasses))
labels.shape
labels[np.arange(len(df)), df['type'].values] = 1 # one-hot encoded target tensor

# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 
dataset.take(1)

dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

print("Train size: " + str(train_size))

20
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63784 entries, 0 to 63783
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   report  63784 non-null  object
 1   type    63784 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 996.8+ KB


generate training data


0it [00:00, ?it/s]

Train size: 3188


### **Model**

In [ ]:
# Setup BERT model
####################################
from transformers import TFBertModel
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(numberOfClasses, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
# Train model
####################################

hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20
)

Epoch 1/20
3188/3188 [==============================] - 336s 102ms/step - loss: 0.7223 - accuracy: 0.7831 - val_loss: 0.4919 - val_accuracy: 0.8578
Epoch 2/20
3188/3188 [==============================] - 326s 102ms/step - loss: 0.5003 - accuracy: 0.8429 - val_loss: 0.4587 - val_accuracy: 0.8699
Epoch 3/20
3188/3188 [==============================] - 324s 102ms/step - loss: 0.4395 - accuracy: 0.8596 - val_loss: 0.4139 - val_accuracy: 0.8810
Epoch 4/20
3188/3188 [==============================] - 324s 102ms/step - loss: 0.3803 - accuracy: 0.8770 - val_loss: 0.3804 - val_accuracy: 0.8912
Epoch 5/20
3188/3188 [==============================] - 324s 102ms/step - loss: 0.3250 - accuracy: 0.8956 - val_loss: 0.3467 - val_accuracy: 0.9030
Epoch 6/20
3188/3188 [==============================] - 324s 102ms/step - loss: 0.2742 - accuracy: 0.9101 - val_loss: 0.3479 - val_accuracy: 0.9074
Epoch 7/20
3188/3188 [==============================] - 324s 102ms/step - loss: 0.2284 - accuracy: 0.9274 - val_

In [ ]:
sentiment_model.save('top15_vul_dataset_nvd_only_vulBERT_v1')

### **Prediction**

In [ ]:
sentiment_model = tf.keras.models.load_model('vulBERT_v1')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = input('Enter movie review here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data, classes)
print(f"Predicted Sentiment: {result}")

Enter movie review here: kmMail does not sufficiently sanitize HTML and script code from the body of e-mail messages.  As a result, an attacker may send a malicious message to a user of kmMail that includes arbitrary HTML and script code.This may allow an attacker to steal cookie-based authentication credentials from users of the webmail system.  Other attacks are also possible.
1/1 [==============================] - 3s 3s/step
Predicted Sentiment: CWE-79
